# Using Cerebellar Architecture to Solve the Three-Body Problem

As an example, the initial data from the study is used:
[A remarkable periodic solution of the three-body problem in the case of equal masses](https://arxiv.org/pdf/math/0011268.pdf)

In [24]:
import pyglet, random, math, os, sys, numpy as np, copy
from pyglet.window import key, FPSDisplay
from pyglet import image, shapes

# begin coordinat left down corner
window = pyglet.window.Window(800,500,"Three bodys",resizable = False)
batch = pyglet.graphics.Batch()
fps_display = FPSDisplay(window)
fps_display.label = pyglet.text.Label(font_name='Arial', font_size=12, x=20, y=20, color = (0,0,0,200))
label = pyglet.text.Label('A.Nurutdinov (c)', font_name='Arial', font_size=12, x=100, y=20, color = (0,0,0,200))
pyglet.gl.glClearColor(1,1,1,1)

# ===================== Parallel faber =========================== 
# The values are obtained from Chenciner & Montgomery (2000)
# https://arxiv.org/pdf/math/0011268.pdf
PFG  = 1.0               # Gravitational constant
PFm  = np.array([1, 1, 1])   # mass size array 

PFx  = np.array([ 0.97000436  , -0.97000436  ,  0         ])   # x coordinates of each of the bodies
PFvx = np.array([ 0.93240737/2,  0.93240737/2, -0.93240737])   # x speed of each body (first derivative of)
PFax = np.array([ .0         ,  .0           ,  .0        ])   # x acceleration of each body (second derivative of)

PFy  = np.array([-0.24308753  , 0.24308753  ,  0         ])   # y coordinates of each of the bodies
PFvy = np.array([ 0.86473146/2, 0.86473146/2, -0.86473146])   # y speed of each body (first derivative of)
PFay = np.array([ .0          , .0          ,  .0        ])   # y acceleration of each body (second derivative of)

ON = False  # running the model 
# ======================= Golgi Cell ============================= 
def GCf(m1,m2,x1,y1,x2,y2):
    dx = x2-x1; dy = y2-y1
    r = math.sqrt(dx*dx+dy*dy)
    return PFG*m2*dx/(r*r*r)
# ======================= prepare procedure =============================
BodyBatch = []  # to save the trajectory 
Color = [(100,100,250),(100,200,100),(200,100,100)]  # blue, green, red
Body = [0,0,0]
for i in range(3):
    Body[i] = shapes.Circle(x=PFx[i], y=PFy[i], radius=15, color=Color[i], batch=batch)
# ======================= drawing procedure =============================
@window.event
def on_draw():
    window.clear()
    fps_display.draw()
    batch.draw()
    #label.draw()
# ======================= mouse press ============================
@window.event
def on_mouse_press(x, y, button, modifiers):
    global ON
    if button == pyglet.window.mouse.LEFT:
        ON = True
# ======================= update procedure ============================
def update(dt):  
    global Body,BodyBatch, ON
    if ON:
        for l in range(300):
            PFax[0] = GCf(PFm[0],PFm[1], PFx[0],PFy[0],PFx[1],PFy[1]) + GCf(PFm[0],PFm[2], PFx[0],PFy[0],PFx[2],PFy[2])
            PFay[0] = GCf(PFm[0],PFm[1], PFy[0],PFx[0],PFy[1],PFx[1]) + GCf(PFm[0],PFm[2], PFy[0],PFx[0],PFy[2],PFx[2])

            PFax[1] = GCf(PFm[1],PFm[0], PFx[1],PFy[1],PFx[0],PFy[0]) + GCf(PFm[1],PFm[2], PFx[1],PFy[1],PFx[2],PFy[2])
            PFay[1] = GCf(PFm[1],PFm[0], PFy[1],PFx[1],PFy[0],PFx[0]) + GCf(PFm[1],PFm[2], PFy[1],PFx[1],PFy[2],PFx[2])

            PFax[2] = GCf(PFm[2],PFm[0], PFx[2],PFy[2],PFx[0],PFy[0]) + GCf(PFm[2],PFm[1], PFx[2],PFy[2],PFx[1],PFy[1])
            PFay[2] = GCf(PFm[2],PFm[0], PFy[2],PFx[2],PFy[0],PFx[0]) + GCf(PFm[2],PFm[1], PFy[2],PFx[2],PFy[1],PFx[1])

            for i in range(3):
                PFvx[i] += PFax[i]*dt/1000; PFx[i] += PFvx[i]*dt/1000
                PFvy[i] += PFay[i]*dt/1000; PFy[i] += PFvy[i]*dt/1000
        for i in range(3):        
            BodyBatch.append(shapes.Circle(400+PFx[i]*320, 250+PFy[i]*350, 2, color=Color[i], batch=batch))
            Body[i].x = 400+PFx[i]*320; Body[i].y = 250+PFy[i]*350
        #label.text = str(PFay[0])
    
# =========================== MAIN LOOP ============================
if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/60)
    pyglet.app.run()
    pyglet.clock.unschedule(update)   # necessary to restart the countdown of time intervals
#========================================================================